In [2]:
# web scrape frame work to get data from MetaCritic.com

In [ ]:
#below import all the packages we need.

In [5]:
import requests
import bs4
import json
import time

In [8]:
# loading a dynamic web page. beautiful soup deals with static html web pages
# but for dynamic pages that use javascript at run time to update we need to 
#use the below method to get this data

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os

# Instantiate an Options object
# and add the "--headless" argument
opts = Options()
opts.add_argument(" --headless")

# Set the location of the webdriver
chrome_driver = os.getcwd() +"\\chromedriver.exe"
# Instantiate a webdriver
driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)

In [3]:
#absolute movie path 
base_url = "https://www.metacritic.com/browse/movies/genre/metascore/{}?view=detailed"

#string array list of url's to scrape
relative_url_paths = []
relative_url_paths.insert(0, "action")
relative_url_paths.insert(1, "adventure")
relative_url_paths.insert(2, "animation")
relative_url_paths.insert(3, "biography")
relative_url_paths.insert(4, "comedy")
relative_url_paths.insert(5, "crime")
relative_url_paths.insert(6, "documentary")
relative_url_paths.insert(7, "drama")
relative_url_paths.insert(8, "family")
relative_url_paths.insert(9, "fantasy")
relative_url_paths.insert(10, "film-noir")
relative_url_paths.insert(11, "history")
relative_url_paths.insert(12, "horror")
relative_url_paths.insert(13, "music")
relative_url_paths.insert(14, "musical")
relative_url_paths.insert(15, "mystery")
relative_url_paths.insert(16, "news")
relative_url_paths.insert(17, "reality-tv")
relative_url_paths.insert(18, "romance")
relative_url_paths.insert(19, "sci-fi")
relative_url_paths.insert(20, "short")
relative_url_paths.insert(21, "sport")
relative_url_paths.insert(22, "thriller")
relative_url_paths.insert(23, "war")
relative_url_paths.insert(24, "western")

In [7]:
#absolute movie path 
base_url = "https://www.metacritic.com/browse/tv/genre/metascore/{}?view=detailed"

#string array list of TV url's to scrape
relative_url_paths = []
relative_url_paths.insert(0, "actionadventure")
relative_url_paths.insert(1, "animation")
relative_url_paths.insert(2, "arts")
relative_url_paths.insert(3, "business")
relative_url_paths.insert(4, "comedy")
relative_url_paths.insert(5, "documentary")
relative_url_paths.insert(6, "drama")
relative_url_paths.insert(7, "educational")
relative_url_paths.insert(8, "eventsspecials")
relative_url_paths.insert(9, "fantasy")
relative_url_paths.insert(10, "foodcooking")
relative_url_paths.insert(11, "gameshow")
relative_url_paths.insert(12, "healthlifestyle")
relative_url_paths.insert(13, "horror")
relative_url_paths.insert(14, "kids")
relative_url_paths.insert(15, "moviemini-series")
relative_url_paths.insert(16, "music")
relative_url_paths.insert(17, "news")
relative_url_paths.insert(18, "newsdocumentary")
relative_url_paths.insert(19, "reality")
relative_url_paths.insert(20, "science")
relative_url_paths.insert(21, "sciencefiction")
relative_url_paths.insert(22, "soap")
relative_url_paths.insert(23, "sports")
relative_url_paths.insert(24, "suspense")
relative_url_paths.insert(25, "talkinterview")

In [9]:
# for each relative url load the page resorce
for relative_path in relative_url_paths:
    # create the master list
    master_movies_list = []
    
    print("setting up driver")
    # Instantiate an Options object
    # and add the "--headless" argument
    opts = Options()
    opts.add_argument(" --headless")

    # Set the location of the webdriver
    chrome_driver = os.getcwd() +"\\chromedriver.exe"
    # Instantiate a webdriver
    driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)
    #from the driver that was initiated earlier:
    # Load the HTML page passing in the relative url to the base path.
    driver.get(base_url.format(relative_path))
    
    print("Driver complete, starting: " + relative_path)

    # Put the page source into a variable and create a beautiful soup object from it
    soup_file = driver.page_source
    soup = bs4.BeautifulSoup(soup_file)

    # select all table rows and save to a variable
    movies = soup.select('.clamp-list > tbody > tr')
    
    # create the key values pair list
    key_value_pairs_list = []

    for movie in movies:
        try:
            # check not empty because there is a empty table row at every second index.
            # continue iterating to the next item.
            if (movie.select(".clamp-summary-wrap > .title > h3") == []):
                continue
            title = movie.select(".clamp-summary-wrap > .title > h3")[0].text.strip()
            rank = movie.select(".clamp-summary-wrap > .title.numbered")[0].text.strip()
            release_date_and_rating = movie.select(".clamp-summary-wrap > .clamp-details")[0].text.strip()
            description = movie.select(".clamp-summary-wrap > .summary")[0].text.strip()
            meta_score = movie.select(".clamp-summary-wrap > .browse-score-clamp > .clamp-metascore")[0].text.strip()
            user_score = movie.select(".clamp-summary-wrap > .browse-score-clamp > .clamp-userscore")[0].text.strip()
            image_url = movie.select(".clamp-image-wrap > a > img")[0]['src']
    
            #create a key_value_pairs list object
            key_value_pairs = {}
        
            # set the key value pair attributes.
            key_value_pairs['title'] = title
            key_value_pairs['description'] = description
            key_value_pairs['releaseDateAndRating'] = release_date_and_rating
            key_value_pairs['meta_score'] = meta_score
            key_value_pairs['user_score'] = user_score
            key_value_pairs['list_rank'] = rank
            key_value_pairs['image_url'] = image_url
    
            #append the key_values_pairs object to the key_value_pairs list 
            key_value_pairs_list.append(key_value_pairs)
        
        except AttributeError as ex:
            print('Error:', ex)
        
    # create a new master object to append each key_values_pair list 
    # along with the asscociated category header.
    master_object = {}

    #set the category header in the newly created master_object
    master_object["categoryHeader"] = "All Time Great " + relative_path + " Tv Shows"
    master_object["listItems"] = key_value_pairs_list

    #append master_object to the master_list
    master_movies_list.append(master_object)
    
    # time out for 30 seconds to avoid being detected as a bot before we head onto the next url page
    print("Sleeping 15 seconds...finished: " + relative_path)
    # Close the browser
    driver.quit()
    time.sleep(15)
    
    #save each category to a new JSON file
    #create the object json will convert
    convert_object = {}
    convert_object['contentList'] = master_movies_list

    # (was after loop - now at the end of each loop)
    #convert master_list into a json file format and save file:
    f = open(relative_path + '_tv.json', 'w')
    f.write(json.dumps(convert_object))
    f.close()
    print("Finished: file saved (" + relative_path + '_tv.json)')

setting up driver
Driver complete, starting: actionadventure
Sleeping 15 seconds...finished: actionadventure
Finished: file saved (actionadventure_tv.json)
setting up driver
Driver complete, starting: animation
Sleeping 15 seconds...finished: animation
Finished: file saved (animation_tv.json)
setting up driver
Driver complete, starting: arts
Sleeping 15 seconds...finished: arts
Finished: file saved (arts_tv.json)
setting up driver
Driver complete, starting: business
Sleeping 15 seconds...finished: business
Finished: file saved (business_tv.json)
setting up driver
Driver complete, starting: comedy
Sleeping 15 seconds...finished: comedy
Finished: file saved (comedy_tv.json)
setting up driver
Driver complete, starting: documentary
Sleeping 15 seconds...finished: documentary
Finished: file saved (documentary_tv.json)
setting up driver
Driver complete, starting: drama
Sleeping 15 seconds...finished: drama
Finished: file saved (drama_tv.json)
setting up driver
Driver complete, starting: educ